In [10]:
"""
品詞のみ抽出して結合
"""

def pick_hinshi(words,m):
  output = []
  parsed_text = m.parseToNode(words)
  
  while parsed_text:
    node = parsed_text.feature.split(',')
    output.append(";".join(node[0:3]))
    parsed_text = parsed_text.next
  
  return "_".join(output)

In [ ]:
# 出現語彙の計測
tmp = sazi_data[~sazi_data["amount"].str.contains(';')]
data_rest = tmp["quantity"].value_counts()
data_rest.head(50)

In [ ]:
data_hinshi = tmp["quantity"].progress_apply(pick_hinshi,m=mecab)
data_hinshi.value_counts().head(50)

In [ ]:
print(" ".join(tmp["quantity"].values.tolist()[0:10]))


In [12]:
# wordcloudで可視化
from wordcloud import WordCloud

text = WordCloud.__doc__
wc = WordCloud(width=480, height=320,background_color="white",font_path=".//SourceHanSerifK-Light.otf")
wc.generate(" ".join(tmp["quantity"].values.tolist()))
wc.to_file('ingredient_amount.png')

In [1]:
import pandas as pd

check = pd.read_csv("./data/exchanged_data_rakuten.csv")
print(len(check[check["digit"] == 0.])/len(check))

0.19862010215961065


In [4]:
# 食材ベクトルの生成

import json
import numpy as np
import pandas as pd

ingredients_vec = {}
with open("./data/recipe_ingredients_rakuten.json","r") as f:
    ingredients_data = json.load(f)

# 変換表の読み込み
exchange_data = pd.read_csv("./data/exchange_complete.csv")

for recipe in ingredients_data.items():
    tmp = np.zeros(499,dtype="float")
    for ingre in recipe[1].items():
        match_ingre = exchange_data.query('name == "{}"'.format(ingre[0]))
        tmp[match_ingre[0:1].kind_index] = ingre[1]
    ingredients_vec[recipe[0]] = tmp

np.save("./rakuten_content_vec.npy",ingredients_vec)

In [ ]:
#　クックパッドコンテンツデータの生成
import pandas as pd

content = pd.read_csv("./data/taken_recipes.csv")
content.head()

In [8]:
cookpad_content = {}
with open("./data/cookpad_recipe_ingredients.json","r") as f:
    ingredients_data = json.load(f)

for recipe in ingredients_data.items():
    cookpad_content[recipe[0]] = content.query('recipe_id=="{}"'.format(recipe[0])).iat[0,2]

In [9]:
with open("./cookpad_content.json","w") as f:
    f.write(json.dumps(cookpad_content,ensure_ascii=False,indent=2))